In [14]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from path import Path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

In [15]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [16]:
file_path = Path("../../data/cleandata/1hot_merged_post_ICO.csv")
df = pd.read_csv(file_path)

In [17]:
df.fillna(0, inplace=True)

In [18]:
X = df.drop(columns = ['Ticker','Name', 'success', 'Longevity'])
y = df["success"].values

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
from collections import Counter
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({0.0: 392, 1.0: 392})

In [11]:
results = []
activations = ['elu', 'relu', 'softmax', 'selu', 'softplus', 'softsign', 'tanh', 'sigmoid', 'hard_sigmoid', 'exponential', 'linear']
optimizers = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
loss_functions = ['mean_squared_error',
                  'mean_absolute_error',
                 'mean_squared_logarithmic_error',
                 'squared_hinge',
                 'hinge',
                 'categorical_hinge',
                 'logcosh',
                 'huber_loss',
                 'binary_crossentropy',
                 'kullback_leibler_divergence',
                 'poisson',
                 ]

In [13]:
for activator in activations:
    for activator2 in activations:
        for final_activator in activations:
            for optimizer in optimizers:    
                for loss_function in loss_functions:

                    nn = Sequential()

                    nn.add(Dense(units=8, input_dim=94, activation=activator))

                    nn.add(Dense(units=10, activation=activator2))

                    # Output layer
                    nn.add(Dense(units=1, activation=final_activator))

                    # Compile the model
                    nn.compile(loss=loss_function, optimizer=optimizer, metrics=["mse"])

                    # Fit the model
                    model = nn.fit(X_resampled, y_resampled, epochs=10, verbose=0)

                    predicted = nn.predict(X_test_scaled)

                    model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

                    result = {
                        'combo':[activator, activator2, final_activator, loss_function, optimizer],
                        'loss' : model_loss,
                        'accuracy' : model_accuracy,
                    }

                    results.append(result)


135/1 - 0s - loss: 0.2163 - mse: 0.1936
135/1 - 0s - loss: 0.4933 - mse: 0.3383
135/1 - 0s - loss: 0.0223 - mse: 0.2530
135/1 - 0s - loss: 0.5954 - mse: 0.4193
135/1 - 0s - loss: 0.6280 - mse: 0.6434
135/1 - 0s - loss: 0.9312 - mse: 0.4816
135/1 - 0s - loss: 0.0956 - mse: 0.2135
135/1 - 0s - loss: 0.1396 - mse: 0.1939
135/1 - 0s - loss: 0.2406 - mse: 0.5418
135/1 - 0s - loss: 0.1194 - mse: 1.6473
135/1 - 0s - loss: nan - mse: 0.8414
135/1 - 0s - loss: 0.0806 - mse: 0.1043
135/1 - 0s - loss: 0.4130 - mse: 0.1815
135/1 - 0s - loss: 0.0101 - mse: 0.3360
135/1 - 0s - loss: 0.5633 - mse: 0.4693
135/1 - 0s - loss: 0.6877 - mse: 0.6574
135/1 - 0s - loss: 0.4670 - mse: 0.8257
135/1 - 0s - loss: 0.0660 - mse: 0.1606
135/1 - 0s - loss: 0.0729 - mse: 0.1205
135/1 - 0s - loss: 0.2858 - mse: 0.2742
135/1 - 0s - loss: 0.0033 - mse: 0.5167
135/1 - 0s - loss: nan - mse: 0.9130
135/1 - 0s - loss: 0.1559 - mse: 0.2798
135/1 - 0s - loss: 0.6501 - mse: 0.6701
135/1 - 0s - loss: 0.0504 - mse: 0.1789
135/1 

KeyboardInterrupt: 

In [ ]:
x = pd.DataFrame(results)

In [ ]:
fsd = x.sort_values('loss', ascending=True)

In [ ]:
fsd[:5]

In [ ]:
rfgc  = x.sort_values('accuracy', ascending=False)
rfgc[:5]

In [22]:
nn = Sequential()

nn.add(Dense(units=8, input_dim=94, activation='softplus'))

nn.add(Dense(units=10, activation='softplus'))

nn.add(Dense(units=10, activation='exponential'))

# Output layer
nn.add(Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss='kullback_leibler_divergence', optimizer='Nadam', metrics=["mse"])

# Fit the model
model = nn.fit(X_train_scaled, y_train, epochs=1000, verbose=0)

predicted = nn.predict(X_test_scaled)

model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

135/1 - 0s - loss: -1.4835e-06 - mse: 0.9761
Normal Neural Network - Loss: -1.355151342371149e-06, Accuracy: 0.9760874509811401


In [23]:
nn_json = nn.to_json()

file_path = Path("./mina_nn3_10k_new_df.json")

with open(file_path, "w") as json_file:
    json_file.write(nn_json)

# Save weights
file_path = Path("./mina_nn3_10k_new_df.h5")
nn.save_weights(file_path)

In [24]:
file_path2 = Path("../../data/cleandata/1hot_merged_360_ICO.csv")
ico_360 = pd.read_csv(file_path2)

In [25]:
ico_360.head()

,Name,Ticker,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,...,platform_Hyperledger,platform_Litecoin,platform_NEM,platform_NEO,platform_Nxt,platform_Omni,platform_Own Blockchain,platform_Scrypt,platform_Stellar,platform_Waves
0,Algorand,ALGO,46,10,0.538655,0.062863,0.040765,0.896412,51,92,...,0,0,0,0,0,0,0,0,0,0
1,Fetch.ai,FET,61,213,0.582417,0.083500,0.040000,0.876333,6,351,...,0,0,0,0,0,0,0,0,0,0
2,Bitbook Gambling,BXK,43,14,0.733514,0.125770,0.046740,0.827460,100,230,...,0,0,0,0,0,0,0,0,0,0
3,BQT,BQTX,181,0,-0.177867,0.006333,0.011333,0.982333,3,243,...,0,0,0,0,0,0,0,0,0,0
4,PlayChip,PLA,7,152,-0.016261,0.027611,0.906611,0.065667,18,353,...,0,0,0,0,0,0,0,0,0,0


In [26]:
X = ico_360.drop(columns = ['Ticker','Name', 'success', 'Longevity'])
X_scaler = StandardScaler()
X_scaler.fit(X)
X_scaled = X_scaler.transform(X)
predictions = nn.predict(X_scaled)

In [43]:
score = predictions
score

array([[0.99997985],
       [0.9988574 ],
       [0.9998008 ],
       [0.9996624 ],
       [1.        ],
       [0.9997932 ],
       [0.999707  ],
       [0.9999894 ],
       [0.9999571 ],
       [0.99982303],
       [0.9999626 ],
       [0.99999964],
       [0.99999905],
       [0.9996199 ],
       [0.99998915],
       [0.9999825 ],
       [0.9999995 ],
       [0.9998479 ],
       [0.9998182 ],
       [0.9964489 ],
       [0.99321705],
       [0.99704707],
       [1.        ],
       [0.99991375],
       [0.99566555],
       [0.9999595 ],
       [0.99999976],
       [       nan],
       [       nan],
       [0.99979323],
       [0.9989254 ],
       [0.99589026],
       [0.99998224],
       [0.9987167 ],
       [0.9973316 ],
       [0.9999777 ]], dtype=float32)

In [31]:
results = []
for s in predictions:
    results.append(*s)
results

[0.99997985,
 0.9988574,
 0.9998008,
 0.9996624,
 1.0,
 0.9997932,
 0.999707,
 0.9999894,
 0.9999571,
 0.99982303,
 0.9999626,
 0.99999964,
 0.99999905,
 0.9996199,
 0.99998915,
 0.9999825,
 0.9999995,
 0.9998479,
 0.9998182,
 0.9964489,
 0.99321705,
 0.99704707,
 1.0,
 0.99991375,
 0.99566555,
 0.9999595,
 0.99999976,
 nan,
 nan,
 0.99979323,
 0.9989254,
 0.99589026,
 0.99998224,
 0.9987167,
 0.9973316,
 0.9999777]

In [32]:
ico_360["Viability"] = results

In [33]:
ico_360.sort_values('Viability', ascending = False)

,Name,Ticker,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,...,platform_Litecoin,platform_NEM,platform_NEO,platform_Nxt,platform_Omni,platform_Own Blockchain,platform_Scrypt,platform_Stellar,platform_Waves,Viability
4,PlayChip,PLA,7,152,-0.016261,0.027611,0.906611,0.065667,18,353,...,0,0,0,0,0,0,0,0,0,1.000000
22,GoldFund,GFUN,61,60,-0.690800,0.007000,0.020000,0.973000,1,289,...,1,0,0,0,0,0,0,0,0,1.000000
26,Advanced Inte...,AIB,92,44,0.950900,0.059000,0.049000,0.892000,1,76,...,0,0,0,0,0,0,0,0,0,1.000000
11,HashCoin,HSC,90,0,-0.066400,0.013500,0.017000,0.969500,2,215,...,0,0,0,0,0,0,0,0,0,1.000000
16,Bittwatt,BWT,65,90,0.226300,0.006000,0.000000,0.994000,1,147,...,0,0,0,0,0,0,0,0,0,1.000000
12,Mindexcoin,MIC,14,9,-0.514596,0.060261,0.891826,0.047913,23,336,...,0,0,0,0,0,0,0,0,0,0.999999
7,Cryptocean,CRON,115,61,0.321625,0.019000,0.067250,0.913750,4,326,...,0,0,0,0,0,0,0,0,0,0.999989
14,BonusCloud,BXC,59,26,0.127433,0.015000,0.013000,0.972000,3,222,...,0,0,0,0,0,0,0,0,0,0.999989
15,Azbit,AZ,211,60,0.547589,0.073222,0.025889,0.900667,9,139,...,0,0,0,0,0,0,0,0,0,0.999982
32,IOU,IOUX,121,14,0.875250,0.133600,0.051800,0.814700,10,320,...,0,0,0,0,0,0,0,0,0,0.999982
